# **Preparation**

## Import library

In [14]:
# !pip install -q opencv-python

In [15]:
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import sys
import tempfile
import tqdm
import math
import numpy.linalg as LA

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Code to run pose estimation using MoveNet

In [16]:
# # Download model from TF Hub and check out inference code from GitHub
# # Model movenet
# !wget -q -O movenet_thunder.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite

# # Github tensorflow (có chứa thư viện utils, data, ml...)
# !git clone https://github.com/tensorflow/examples.git


#### Load MoveNet Thunder model

In [17]:
from pose_estimation import utils
from pose_estimation.data import BodyPart
from pose_estimation.ml import Movenet
movenet = Movenet('movenet_thunder')

# Gọi hàm nhận dạng của movenet nhiều lần để cải thiện độ chính xác nhận dạng khung xương
def detect(input_tensor, inference_count=3):
  """Runs detection on an input image.
 
  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
    inference_count: Số lần lặp.
 
  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape
 
  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)
 
  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(), 
                            reset_crop_region=False)


  return person

In [18]:
imgPath = "./Image/_/output4_0330.png"
image = tf.io.read_file(imgPath)
img = tf.io.decode_jpeg(image)
pers = detect(img)

#### Trực quan hoá kết quả nhận dạng khung xương

In [19]:
def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.
 
  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.
 
  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])
  
  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)
 
  if close_figure:
    plt.close(fig)
 
  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np

#### Code to load the images, detect pose landmarks and save them into a CSV file

In [20]:
class MoveNetPreprocessor(object): 
  def __init__(self,
               images_in_folder,
               images_out_folder,
               csvs_out_path):
    """Creates a preprocessor to detection pose from images and save as CSV.

    Args:
      images_in_folder: Path to the folder with the input images. It has structure:
        
        train/test
        |__ humpbacked
            |______ 00000128.jpg
            |______ 00000181.bmp
            |______ ...
        |__ leaning back
            |______ 00000243.jpg
            |______ 00000306.jpg
            |______ ...
        ...

      images_out_folder: Path to write detected landmarks

      csvs_out_path: Path to write the CSV containing the detected landmark
        coordinates and label of each image that can be used to train a pose
        classification model.
    """
    self._images_in_folder = images_in_folder
    self._images_out_folder = images_out_folder
    self._csvs_out_path = csvs_out_path
    self._messages = []

    # Create a temp dir to store the pose CSVs per class
    self._csvs_out_folder_per_class = tempfile.mkdtemp()
 
    # Get list of pose classes
    self._pose_class_names = sorted(
        [n for n in os.listdir(self._images_in_folder)]
        )
    
  def process(self, per_pose_class_limit=None, detection_threshold=0.1):
    """Preprocesses images in the given folder.
    Args:
      per_pose_class_limit: Number of images to load in each class directory. 
      detection_threshold: Only keep images with all landmark confidence score
        above this threshold.
    """
    # Loop through the class directory and preprocess its images
    for pose_class_name in self._pose_class_names:
      print('Preprocessing', pose_class_name, file=sys.stderr)

      # Paths for the pose class.
      images_in_folder = os.path.join(self._images_in_folder, pose_class_name)    # Tạo đường dẫn
      images_out_folder = os.path.join(self._images_out_folder, pose_class_name)
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  pose_class_name + '.csv')
      if not os.path.exists(images_out_folder):
        os.makedirs(images_out_folder)
 
      # Detect landmarks in each image and write it to a CSV file
      with open(csv_out_path, 'w') as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file, 
                                    delimiter=',', 
                                    quoting=csv.QUOTE_MINIMAL)
        # Get list of images
        image_names = sorted(
            [n for n in os.listdir(images_in_folder) if not n.startswith('.')])
        if per_pose_class_limit is not None:
          image_names = image_names[:per_pose_class_limit]

        valid_image_count = 0
 
        # Detect pose landmarks from each image
        for image_name in tqdm.tqdm(image_names):
          image_path = os.path.join(images_in_folder, image_name)

          try:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
          except:
            self._messages.append('Skipped ' + image_path + '. Invalid image.')
            continue
          else:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
            image_height, image_width, channel = image.shape
          
          # Skip images that isn't RGB because Movenet requires RGB images
          if channel != 3:
            self._messages.append('Skipped ' + image_path +
                                  '. Image isn\'t in RGB format.')
            continue
          person = detect(image)
          
          # Save landmarks if all landmarks were detected
          min_landmark_score = min(
              [keypoint.score for keypoint in person.keypoints])
          should_keep_image = min_landmark_score >= detection_threshold
          if not should_keep_image:
            self._messages.append('Skipped ' + image_path +
                                  '. No pose was confidentlly detected.')
            continue

          valid_image_count += 1

          # Draw the prediction result on top of the image for debugging later
          output_overlay = draw_prediction_on_image(
              image.numpy().astype(np.uint8), person, 
              close_figure=True, keep_input_size=True)
        
          # Write detection result into an image file
          output_frame = cv2.cvtColor(output_overlay, cv2.COLOR_RGB2BGR)
          cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)
        
          # Get landmarks and scale it to the same size as the input image
          pose_landmarks = np.array(
              [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
                for keypoint in person.keypoints],
              dtype=np.float32)

          # Write the landmark coordinates to its per-class CSV file
          coordinates = pose_landmarks.flatten().astype(str).tolist()
          csv_out_writer.writerow([image_name] + coordinates)

        if not valid_image_count:
          raise RuntimeError(
              'No valid images found for the "{}" class.'
              .format(pose_class_name))
      
    # Print the error message collected during preprocessing.
    print('\n'.join(self._messages))

    # Combine all per-class CSVs into a single output file
    all_landmarks_df = self._all_landmarks_as_dataframe()
    all_landmarks_df.to_csv(self._csvs_out_path, index=False)

  def class_names(self):
    """List of classes found in the training dataset."""
    return self._pose_class_names
  
  def _all_landmarks_as_dataframe(self):
    """Merge all per-class CSVs into a single dataframe."""
    total_df = None
    for class_index, class_name in enumerate(self._pose_class_names):
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  class_name + '.csv')
      per_class_df = pd.read_csv(csv_out_path, header=None)
      
      # Add the labels
      per_class_df['class_no'] = [class_index]*len(per_class_df)
      per_class_df['class_name'] = [class_name]*len(per_class_df)

      # Append the folder name to the filename column (first column)
      per_class_df[per_class_df.columns[0]] = (os.path.join(class_name, '') 
        + per_class_df[per_class_df.columns[0]].astype(str))

      if total_df is None:
        # For the first class, assign its data to the total dataframe
        total_df = per_class_df
      else:
        # Concatenate each class's data into the total dataframe
        total_df = pd.concat([total_df, per_class_df], axis=0)
 
    list_name = [[bodypart.name + '_x', bodypart.name + '_y', 
                  bodypart.name + '_score'] for bodypart in BodyPart] 
    
    # Đổi tên cột dataframe
    header_name = []
    for columns_name in list_name:
      header_name += columns_name
    header_name = ['file_name'] + header_name
    header_map = {total_df.columns[i]: header_name[i] 
                  for i in range(len(header_name))}
 
    total_df.rename(header_map, axis=1, inplace=True)

    return total_df

#### Test pose estimate

# **Part 1: Preprocess the input images**

### Upload your own pose dataset

In [21]:
dataset_is_split = True #@param ["False", "True"] {type:"raw"}


  If you've already split your dataset into train and test sets, then set `dataset_is_split` to **True**. 
  That is, your images folder must include "train" and "test" directories like this:
```
    sittingPoses/
    |__ train/
        |__ humpbacked/
            |______ 00000128.jpg
            |______ ...
        |__ leaning back/
            |______ 00000180.jpg
            |______ ...
    |__ test/
        |__ leaning back/
            |______ 00000181.jpg
            |______ ...
```

Or, if your dataset is NOT split yet, then set
`dataset_is_split` to **False** and we'll split it up based
on a specified split fraction. That is, your uploaded images
folder should look like this:

```    
    sittingPoses/
    |__ humpbacked/
        |______ 00000128.jpg
        |______ 00000181.jpg
        |______ ...
    |__ leaning back/
        |______ 00000243.jpg
        |______ 00000306.jpg
        |______ ...
```

### Phân chia tập dữ liệu

In [22]:
import os
import random
import shutil

def split_into_train_test(images_origin, images_dest, test_split):
  """Splits a directory of sorted images into training and test sets.

  Args:
    images_origin: Path to the directory with your images. This directory
      must include subdirectories for each of your labeled classes.

    images_dest: Path to a directory where you want the split dataset to be
      saved

    test_split: Fraction of data to reserve for test (float between 0 and 1).
  """
  _, dirs, _ = next(os.walk(images_origin))

  TRAIN_DIR = os.path.join(images_dest, 'train')
  TEST_DIR = os.path.join(images_dest, 'test')
  os.makedirs(TRAIN_DIR, exist_ok=True)
  os.makedirs(TEST_DIR, exist_ok=True)

  for dir in dirs:
    # Get all filenames for this dir, filtered by filetype
    filenames = os.listdir(os.path.join(images_origin, dir))
    filenames = [os.path.join(images_origin, dir, f) for f in filenames if (
        f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.bmp'))]
    # Shuffle the files, deterministically
    filenames.sort()
    random.seed(42)
    random.shuffle(filenames)
    # Divide them into train/test dirs
    os.makedirs(os.path.join(TEST_DIR, dir), exist_ok=True)
    os.makedirs(os.path.join(TRAIN_DIR, dir), exist_ok=True)
    test_count = int(len(filenames) * test_split)
    for i, file in enumerate(filenames):
      if i < test_count:
        destination = os.path.join(TEST_DIR, dir, os.path.split(file)[1])
      else:
        destination = os.path.join(TRAIN_DIR, dir, os.path.split(file)[1])
      shutil.copyfile(file, destination)
    print(f'Moved {test_count} of {len(filenames)} from class "{dir}" into test.')
  print(f'Your split dataset is in "{images_dest}"')

In [23]:
# You can leave the rest alone:
dataset_in = './image_new'
if not os.path.isdir(dataset_in):
  raise Exception("dataset_in is not a valid directory")
if dataset_is_split:
  IMAGES_ROOT = dataset_in
else:
  dataset_out = dataset_in + '_split'
  split_into_train_test(dataset_in, dataset_out, test_split=0.2)
  IMAGES_ROOT = dataset_out

**Note:** If you're using `split_into_train_test()` to split the dataset, it expects all images to be PNG, JPEG, or BMP—it ignores other file types.

# Process single dataset

In [24]:
isProcessed = True

In [25]:
images_out_test_folder = 'Nhat_pose'

if isProcessed: pass
else:
    images_in_test_folder = os.path.join(IMAGES_ROOT, 'Nhat')
    csvs_out_test_path = 'N.csv'

    preprocessor = MoveNetPreprocessor(
        images_in_folder=images_in_test_folder,
        images_out_folder=images_out_test_folder,
        csvs_out_path=csvs_out_test_path,
    )

    preprocessor.process(per_pose_class_limit=None)

In [26]:
images_out_test_folder = 'Nhi_pose'

if isProcessed: pass
else:
    images_in_test_folder = os.path.join(IMAGES_ROOT, 'Nhi')
    csvs_out_test_path = 'NH.csv'

    preprocessor = MoveNetPreprocessor(
        images_in_folder=images_in_test_folder,
        images_out_folder=images_out_test_folder,
        csvs_out_path=csvs_out_test_path,
    )

    preprocessor.process(per_pose_class_limit=None)

In [27]:
images_out_test_folder = 'Hoan_pose'

if isProcessed: pass
else:
    images_in_test_folder = os.path.join(IMAGES_ROOT, 'Hoan')
    csvs_out_test_path = 'H.csv'

    preprocessor = MoveNetPreprocessor(
        images_in_folder=images_in_test_folder,
        images_out_folder=images_out_test_folder,
        csvs_out_path=csvs_out_test_path,
    )

    preprocessor.process(per_pose_class_limit=None)

In [28]:
images_out_test_folder = 'Khoi_pose'

if isProcessed: pass
else:
    images_in_test_folder = os.path.join(IMAGES_ROOT, 'Khoi')
    csvs_out_test_path = 'K.csv'

    preprocessor = MoveNetPreprocessor(
        images_in_folder=images_in_test_folder,
        images_out_folder=images_out_test_folder,
        csvs_out_path=csvs_out_test_path,
    )

    preprocessor.process(per_pose_class_limit=None)

# **Part 2: Train a pose classification model that takes the landmark coordinates as input, and output the predicted labels.**

You'll build a TensorFlow model that takes the landmark coordinates and predicts the pose class that the person in the input image performs. The model consists of two submodels:

* Submodel 1 calculates a pose embedding (a.k.a feature vector) from the detected landmark coordinates.
* Submodel 2 feeds pose embedding through several `Dense` layer to predict the pose class.

You'll then train the model based on the dataset that were preprocessed in part 1.

### Load the preprocessed CSVs into `train` and `test` variables

In [29]:
def load_pose_landmarks(csv_path):
  """Loads a CSV created by MoveNetPreprocessor.
  
  Returns:
    X: Detected landmark coordinates and scores of shape (N, 17 * 3)
    y: Ground truth labels of shape (N, label_count)
    classes: The list of all class names found in the dataset
    dataframe: The CSV loaded as a Pandas dataframe features (X) and ground
      truth labels (y) to use later to train a pose classification model.
  """
  # Load the CSV file
  dataframe = pd.DataFrame()
  if type(csv_path) is np.ndarray:
    for path in csv_path:
      dataframe = pd.concat([dataframe, pd.read_csv(path)],
                            axis=0, ignore_index=True)
  else:
    dataframe = pd.read_csv(csv_path)

  df_to_process = dataframe.copy()

  # Drop the file_name columns as you don't need it during training.
  df_to_process.drop(columns=['file_name'], inplace=True)

  # Extract the list of class names
  classes = df_to_process.pop('class_name').unique()

  # Extract the labels
  y = df_to_process.pop('class_no')

  # Convert the input features and labels into the correct format for training.
  X = df_to_process.astype('float64')
  y = keras.utils.to_categorical(y)

  return X, y, classes, dataframe

### Define functions to convert the pose landmarks to a pose embedding (a.k.a. feature vector) for pose classification

Next, convert the landmark coordinates to a feature vector by:
1. Moving the pose center to the origin.
2. Scaling the pose so that the pose size becomes 1
3. Flattening these coordinates into a feature vector

Then use this feature vector to train a neural-network based pose classifier.

In [30]:
def get_center_point(landmarks, left_bodypart, right_bodypart):
    """Calculates the center point of the two given landmarks."""

    left = tf.gather(landmarks, left_bodypart.value, axis=1)
    right = tf.gather(landmarks, right_bodypart.value, axis=1)
    center = left * 0.5 + right * 0.5
    return center


def get_pose_size(landmarks, torso_size_multiplier=2.5):
    """Calculates pose size.

    It is the maximum of two values:
      * Torso size multiplied by `torso_size_multiplier`
      * Maximum distance from pose center to any pose landmark
    """
    # Hips center
    hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP,
                                   BodyPart.RIGHT_HIP)

    # Shoulders center
    shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
                                        BodyPart.RIGHT_SHOULDER)

    # Torso size as the minimum body size
    torso_size = tf.linalg.norm(shoulders_center - hips_center)

    # Pose center
    pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP,
                                       BodyPart.RIGHT_HIP)
    pose_center_new = tf.expand_dims(pose_center_new, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to
    # perform substraction
    pose_center_new = tf.broadcast_to(pose_center_new,
                                      [tf.size(landmarks) // (17*2), 17, 2])

    # Dist to pose center
    d = tf.gather(landmarks - pose_center_new, 0, axis=0,
                  name="dist_to_pose_center")
    # Max dist to pose center
    max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

    # Normalize scale
    pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)

    return pose_size


def feature_pose(landmarks):
    """Normalizes the landmarks translation by moving the pose center to (0,0) and
    scaling it to a constant pose size.
    """
    pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP,
                                BodyPart.RIGHT_HIP)
    pose_center = tf.expand_dims(pose_center, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to perform
    # substraction
    pose_center = tf.broadcast_to(pose_center,
                                [tf.size(landmarks) // (17*2), 17, 2])
    landmarks = landmarks - pose_center

    # Scale the landmarks to a constant pose size
    pose_size = get_pose_size(landmarks)

    xVal = tf.gather(landmarks, BodyPart.RIGHT_KNEE.value, axis=1).numpy()[0]
    tempList = landmarks.numpy().tolist()

    if xVal[0] < 0:
        for coordinate in tempList[0]:
            coordinate[0] = -coordinate[0]
        landmarks = tf.constant(tempList, dtype=np.float32)

    landmarks /= pose_size

    return landmarks


def normalize_drop_score(landmarks_and_scores):
    """Converts the input landmarks into a pose embedding."""
    # Reshape the flat input into a matrix with shape=(17, 3)
    reshaped_inputs = keras.layers.Reshape((17, 3))(landmarks_and_scores)

    # Normalize landmarks 2D
    norm_landmarks = feature_pose(reshaped_inputs[:, :, :2])

    return norm_landmarks


def angle_between_two_vector(a, b):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    inner = np.inner(a, b)
    norms = LA.norm(a) * LA.norm(b)
    cos = inner / norms
    return np.arccos(np.clip(cos, -1.0, 1.0))


def flatten(x):
    return x.numpy().flatten()


def angle_between_three_point(landmarks, bodypart1, bodypart2, bodypart3, isBodyPart=True):
    if isBodyPart:
        bodypart1 = tf.gather(landmarks, bodypart1.value, axis=1)
        bodypart2 = tf.gather(landmarks, bodypart2.value, axis=1)
        bodypart3 = tf.gather(landmarks, bodypart3.value, axis=1)

    v21 = bodypart1 - bodypart2
    v23 = bodypart3 - bodypart2

    # đơn vị radian (góc đúng cả 2 bên)
    return angle_between_two_vector(flatten(v21), flatten(v23))


def feature_angle(landmarks):
    center_ear = get_center_point(
        landmarks, BodyPart.LEFT_EAR, BodyPart.RIGHT_EAR)
    center_shoulder = get_center_point(
        landmarks, BodyPart.LEFT_SHOULDER, BodyPart.RIGHT_SHOULDER)
    center_hip = get_center_point(
        landmarks, BodyPart.LEFT_HIP, BodyPart.RIGHT_HIP)

    angle_ear_shoulder = angle_between_three_point(
        landmarks, center_ear, center_shoulder, center_hip, False)

    angle_left_torso_thighs = angle_between_three_point(
        landmarks, BodyPart.LEFT_SHOULDER, BodyPart.LEFT_HIP, BodyPart.LEFT_KNEE)

    angle_right_torso_thighs = angle_between_three_point(
        landmarks, BodyPart.RIGHT_SHOULDER, BodyPart.RIGHT_HIP, BodyPart.RIGHT_KNEE)

    angle_left_thighs_tibia = angle_between_three_point(
        landmarks, BodyPart.LEFT_HIP, BodyPart.LEFT_KNEE, BodyPart.LEFT_ANKLE)

    angle_right_thighs_tibia = angle_between_three_point(
        landmarks, BodyPart.RIGHT_HIP, BodyPart.RIGHT_KNEE, BodyPart.RIGHT_ANKLE)
    
    backbone_vector = flatten(center_shoulder - center_hip)
    horizontal_backbone_angle = angle_between_two_vector(backbone_vector, (1,0))

    # print('angle between left torso and thighs: ',
    #       np.rad2deg(angle_left_torso_thighs))

    return [horizontal_backbone_angle*2, angle_ear_shoulder, angle_left_torso_thighs, angle_left_thighs_tibia, angle_right_torso_thighs, angle_right_thighs_tibia]


def get_distance(landmarks, left, right):
    left = tf.gather(landmarks, left.value, axis=1)
    right = tf.gather(landmarks, right.value, axis=1)

    vector2 = (right - left).numpy()[0] ** 2

    return math.sqrt(vector2[0] + vector2[1])



In [31]:
def data_preprocess_single(landmarks):
    landmarks = normalize_drop_score(landmarks)
    feature_vector = feature_angle(landmarks)
    feature_vector.append(get_distance(landmarks, BodyPart.LEFT_SHOULDER, BodyPart.RIGHT_SHOULDER))
    pose = feature_pose(landmarks).numpy().tolist()

    unwanted = [BodyPart.LEFT_ELBOW.value, BodyPart.RIGHT_ELBOW.value, BodyPart.LEFT_WRIST.value, BodyPart.RIGHT_WRIST.value]
    for index in sorted(unwanted, reverse = True): 
        del pose[0][index]

    wanted_point = []

    for item in pose[0]:
        for i in item:
            wanted_point.append(i)

    feature_vector.extend(wanted_point)
    # feature_vector = feature_pose(landmarks).numpy().flatten().tolist()
    return feature_vector

def data_preprocess(X: pd.DataFrame):
    data = X.copy()
    listFeature = []
    for landmark in data.values:
        landmark = tf.constant([landmark])
        feature = data_preprocess_single(landmark)
        listFeature.append(feature)

    return pd.DataFrame(listFeature)

### Define a Keras model for pose classification

Our Keras model takes the detected pose landmarks, then calculates the pose embedding and predicts the pose class.

In [32]:
# Define the model
def trainModel(X_train, y_train, X_val, y_val, X_test, y_test, class_names):
    inputs = tf.keras.Input(shape=(33))

    layer = keras.layers.Dense(64, activation=tf.nn.relu6)(inputs)
    layer = keras.layers.Dropout(0.5)(layer)
    layer = keras.layers.Dense(32, activation=tf.nn.relu6)(layer)
    layer = keras.layers.Dropout(0.5)(layer)
    outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)

    model = tf.keras.Model(inputs, outputs)

    model.summary();          # ';' discards the output

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Add a checkpoint callback to store the checkpoint that has the highest
    # validation accuracy.
    checkpoint_path = "weights.best.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                                monitor='val_accuracy',
                                verbose=1,
                                save_best_only=True,
                                mode='max')
    earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                                patience=20)

    # Start training
    history = model.fit(X_train, y_train,
                        epochs=200,
                        batch_size=16,
                        validation_data=(X_val, y_val),
                        callbacks=[checkpoint, earlystopping]);          # ';' discards the output


    # Evaluate the model using the TEST dataset
    loss, accuracy = model.evaluate(X_test, y_test)

    trainLastAccuracy = history.history['accuracy'][len(history.history['accuracy']) - 1]
    
    validateLastAccuracy = history.history['val_accuracy'][len(
        history.history['val_accuracy']) - 1]

    return trainLastAccuracy, validateLastAccuracy, accuracy


In [33]:
X_N, y_N, class_names, test_N = load_pose_landmarks("N.csv")
X_H, y_H, _, test_H = load_pose_landmarks("H.csv")
X_NH, y_NH, _, test_NH = load_pose_landmarks("NH.csv")
X_K, y_K, _, test_K = load_pose_landmarks("K.csv")

In [34]:
X_N = data_preprocess(X_N)
X_H = data_preprocess(X_H)
X_NH = data_preprocess(X_NH)
X_K = data_preprocess(X_K)

In [89]:
data = {
    'N': (X_N, y_N, test_N),
    'NH': (X_NH, y_NH, test_NH),
    'H': (X_H, y_H, test_H),
    'K': (X_K, y_K, test_K)
}

In [92]:
x1 = data['N'][0]
x2 = data['H'][0]

pd.concat([x1, x2])

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,2.844244,3.013107,1.540147,1.903108,0.621909,0.778916,0.117683,0.108308,-0.339890,0.118548,...,-0.039779,-0.002757,0.276874,0.046868,0.133514,-0.202831,0.309958,0.268210,0.147693,0.014572
1,2.820458,3.026063,1.500786,1.879114,0.716957,0.748645,0.120256,0.116248,-0.342812,0.124156,...,-0.037168,-0.005140,0.280734,0.047053,0.162905,-0.196514,0.311576,0.269268,0.150252,0.017793
2,2.822625,3.057042,1.521635,1.910089,0.556597,0.766177,0.124190,0.112191,-0.341347,0.119352,...,-0.038193,-0.002387,0.280875,0.050924,0.115374,-0.207673,0.312702,0.273714,0.143222,0.015914
3,2.782677,3.077987,1.487154,1.900978,0.613443,0.773992,0.113900,0.110391,-0.343788,0.121430,...,-0.034694,-0.003154,0.283073,0.047310,0.144296,-0.205800,0.317767,0.270457,0.155335,0.012616
4,2.856954,3.111899,1.543219,1.918377,0.550465,0.777906,0.125706,0.056135,-0.355262,0.077333,...,-0.038863,-0.001178,0.275181,0.049855,0.112663,-0.213943,0.306586,0.265760,0.149564,0.016095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436,3.246586,2.860245,1.843376,1.699687,1.798919,1.619246,0.023394,0.038162,-0.356457,0.049065,...,-0.010484,0.005032,0.217648,0.041936,0.220164,0.045291,0.196680,0.264197,0.190809,0.280133
1437,3.246227,2.903681,1.849144,1.720219,1.815215,1.642740,0.038747,0.033572,-0.360257,0.044763,...,-0.016356,0.006456,0.216068,0.041750,0.217790,0.049498,0.197130,0.257818,0.191965,0.283643
1438,3.232900,2.941187,1.836271,1.713220,1.810108,1.624108,0.048945,0.031693,-0.359332,0.042829,...,-0.018845,0.007281,0.217570,0.041544,0.215000,0.048396,0.196155,0.258257,0.186733,0.281384
1439,3.225156,2.892892,1.806919,1.655878,1.781077,1.611702,0.026381,0.039487,-0.358326,0.049569,...,-0.011762,0.004621,0.215920,0.036547,0.216760,0.042428,0.189875,0.264228,0.187354,0.280191


In [101]:
y1 = data['N'][1]
y2 = data['H'][1]

y_train = y1.tolist()
y_train.extend(y2.tolist())
y_train = np.array(y_train)
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [102]:
# START CROSS-VALIDATION
lstCsv = np.array(['N', 'NH', 'K', 'H'])
lstAcc = []
for test in lstCsv:
    for validate in lstCsv[lstCsv != test]:
        train = lstCsv[(lstCsv != test) & (lstCsv != validate)]
        print(train[1])

        X_train = pd.concat([data[train[0]][0], data[train[1]][0]])
        y1 = data[train[0]][1]
        y2 = data[train[1]][1]
        y_train = y1.tolist()
        y_train.extend(y2.tolist())
        y_train = np.array(y_train)
        X_val, y_val, _ = data[validate]
        X_test, y_test, df_test = data[test]

        train_acc, val_acc, test_acc = trainModel(
            X_train, y_train, X_val, y_val, X_test, y_test, class_names)
            
        lstAcc.append([train_acc, val_acc, test_acc])
        print([train_acc, val_acc, test_acc])

print(pd.DataFrame([lstAcc]))

H
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 33)]              0         
                                                                 
 dense_6 (Dense)             (None, 64)                2176      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_8 (Dense)             (None, 7)                 231       
                                                                 
Total params: 4,487
Trainable params: 4,487
Non-trainable

In [103]:
lstAcc

[[0.9006121754646301, 0.6696795225143433, 0.6461352705955505],
 [0.9194883108139038, 0.777694582939148, 0.8556763529777527],
 [0.9067763686180115, 0.753643274307251, 0.8454106450080872],
 [0.9218581318855286, 0.7657004594802856, 0.6713229417800903],
 [0.9199225306510925, 0.7178143858909607, 0.8775677680969238],
 [0.9535427689552307, 0.5551699995994568, 0.741988480091095],
 [0.8901429772377014, 0.8164251446723938, 0.6706587076187134],
 [0.9060381054878235, 0.9178307056427002, 0.7732036113739014],
 [0.9540550112724304, 0.6169326901435852, 0.66242516040802],
 [0.8950254321098328, 0.8164251446723938, 0.7349063158035278],
 [0.9642379879951477, 0.7469186782836914, 0.5412907600402832],
 [0.9533588290214539, 0.6422155499458313, 0.6877168416976929]]

In [104]:
a = np.array(['K.csv' 'H.csv'])
type(a) is np.ndarray

True